In [18]:
import pya
import numpy as np

layout = pya.Layout()
top = layout.create_cell('TOP')

l0000 = layout.layer(1000, 0)
l0001 = layout.layer(1000, 1)
l00 = layout.layer(0, 0) # Isolation
l10 = layout.layer(1, 0) # Mesa
l20 = layout.layer(2, 0) # CSL
l30 = layout.layer(3, 0) # ELEC
l40 = layout.layer(4, 0) # ContactHole
l41 = layout.layer(4, 1) # Oxide

##################################  parameters for the mask  ###################################
factor = 1000

mask_size = 10*10**4*factor
mask_bound = pya.Box(0, 0, mask_size, mask_size)
wafer_bound = pya.Path([pya.Point(mask_size/2, mask_size/2)], mask_size, mask_size/2, mask_size/2, True)

led_x = 30*factor
led_y = 100*factor
p_y = 60*factor
n_y = 30*factor

ly_gap = 5*factor
hole_r = 12*factor

##################################  deep etch isolation  ###################################
led = layout.create_cell("LED")
led.shapes(l00).insert(pya.Box(0-2*factor, 0-2*factor, led_x+2*factor, led_y+2*factor))

##################################  LED Cell  ###################################

led.shapes(l10).insert(pya.Box(0, led_y-p_y, led_x, led_y))
led.shapes(l20).insert(pya.Box(ly_gap, led_y-p_y+ly_gap, led_x-ly_gap, led_y-ly_gap))
led.shapes(l30).insert(pya.Box(ly_gap, led_y-p_y+ly_gap, led_x-ly_gap, led_y-ly_gap))
# led.shapes(l30).insert(pya.Box(0, 0, led_x, n_y))
# led.shapes(l30).insert(pya.Path([pya.Point(led_x/2,n_y/2)],30*factor,15*factor,15*factor,True))
# led.shapes(l40).insert(pya.Path([pya.Point(led_x/2,led_y-p_y/2)],2*hole_r,hole_r,hole_r,True))
# led.shapes(l40).insert(pya.Path([pya.Point(led_x/2,n_y/2)],2*hole_r,hole_r,hole_r,True))

radius1 = 15*factor
radius2 = 15*factor
nr_points = 32
angles = np.linspace(0,2*np.pi,nr_points+1)[0:-1]
points1 = []
points2 = []
for angle in angles:
    points1.append(pya.Point(radius1*np.cos(angle),radius1*np.sin(angle)))
    points2.append(pya.Point(radius2*np.cos(angle),radius2*np.sin(angle)))
circle1 = pya.SimplePolygon(points1).moved(15*factor, 15*factor)
circle2 = pya.SimplePolygon(points1).moved(15*factor, 70*factor)
circle3 = pya.SimplePolygon(points2).moved(15*factor, 15*factor)
led.shapes(l40).insert(circle1)
led.shapes(l40).insert(circle2)
led.shapes(l30).insert(circle3)

##################################  LEDs Cell  ###################################
leds = layout.create_cell("LEDs")
leds.insert(pya.CellInstArray(led.cell_index(), pya.Trans(0, 0)))
leds.insert(pya.CellInstArray(led.cell_index(), pya.CplxTrans(1, 45, False, 15*factor, -15*factor*(np.sqrt(2)-1))))
leds.insert(pya.CellInstArray(led.cell_index(), pya.Trans(30*factor, 0)*pya.Trans.R90))
leds.insert(pya.CellInstArray(led.cell_index(), pya.CplxTrans(1, 135, False, 15*factor*(np.sqrt(2)+1), 15*factor)))
leds.insert(pya.CellInstArray(led.cell_index(), pya.Trans(0, 30*factor)*pya.Trans.M0))
leds.insert(pya.CellInstArray(led.cell_index(), pya.CplxTrans(1, -135, False, 15*factor, 15*factor*(np.sqrt(2)+1))))
leds.insert(pya.CellInstArray(led.cell_index(), pya.Trans(0, 30*factor)*pya.Trans.R270))
leds.insert(pya.CellInstArray(led.cell_index(), pya.CplxTrans(1, -45, False, -15*factor*(np.sqrt(2)-1), 15*factor)))

##################################  TOP Cell  ###################################
i = 600
array_x = 300*factor
array_y = 300*factor
# trans = pya.Trans(pya.Point(mask_size/2, mask_size/2))
trans = pya.Trans(pya.Point(5000*factor, 5000*factor))
new_instance = pya.CellInstArray(leds.cell_index(),trans,pya.Vector(array_x, 0 ), pya.Vector(0, array_y), i/2, i/2)
top.insert(new_instance)
top.shapes(l0000).insert(mask_bound)
top.shapes(l0001).insert(wafer_bound)

##################################  NFF: small boxes for each mask layer  ###################################
ly_list = layout.layer_indexes()
for ly in ly_list:
    ly_data = str(layout.get_info(ly))
    # print(ly_data)
    
    if int(ly_data.split('/')[0]) > 0 and int(ly_data.split('/')[0]) <1000 and ly_data.split('/')[1] == '0':
        print(ly_data)
        top.shapes(ly).insert(pya.Box(0, 0, 1*factor, 1*factor))
        top.shapes(ly).insert(pya.Box(mask_size-1*factor, mask_size-1*factor, mask_size, mask_size))

##################################  Get Boolean of Oxide  ###################################
# obj_cell = layout.create_cell("OBJ")
# obj_cell.copy_tree(layout.cell("TOP"))
# idx = obj_cell.cell_index()
# all_mask = obj_cell.flatten(idx)
# oxide = pya.Region(mask_bound) ^ pya.Region(all_mask.shapes(l40))
# top.shapes(l41).insert(oxide)
#############################################################################################

layout.write("test.gds")

1/0
2/0
3/0
4/0


In [12]:
obj_cell = layout.create_cell("OBJ")
obj_cell.copy_tree(layout.cell("TOP"))
idx = obj_cell.cell_index()
all_mask = obj_cell.flatten(idx)

f = open("../mask_lyp.lyp")
lines = f.read()
f.close()

for ly in ly_list:
    ly_data = str(layout.get_info(ly))
    if int(ly_data.split('/')[0]) > 0 and int(ly_data.split('/')[0]) <1000 and ly_data.split('/')[1] == '0':
        obj_ly = all_mask.shapes(ly)
        end_idx = lines.find(ly_data)
        # print(end_idx)
        beg_idx = lines.rfind(">",0,end_idx)
        # print(beg_idx)
        ly_name = lines[beg_idx+1:end_idx-1]
        # print(ly_name)
        newcell = layout.create_cell(ly_data.replace("/","_")+"_"+ly_name)
        newcell.shapes(ly).insert(obj_ly.transform(pya.Trans(mask_size, 0)*pya.Trans.M90))
        # newcell.shapes(l00).insert(mask_bound)
        # newcell.shapes(l01).insert(wafer_bound)
        newcell.write(ly_data.replace("/","_")+"_"+ly_name+".gds")
        layout.delete_cell(newcell.cell_index()) # if not delete cell, duplicated single layer cells will be created.

layout.delete_cell(obj_cell.cell_index()) # if not delete cell, duplicated OBJ cells will be created.
layout.write("test2.gds")
